Using CIFAR 10 dataset to create a basic image classification problem in PyTorch 



Check to see if they want to use the advanced way if so here is link: 
https://github.com/pytorch/examples/blob/master/imagenet/main.py#L167


In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import math 
import shutil
import os


In [2]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5),
                                                                          (0.5, 0.5, 0.5))])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, 
                                        transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)
# how_many_images = trainloader.size


classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified


In [3]:
use_gpu = torch.cuda.is_available()

In [4]:
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self): 
        super(Net, self).__init__()
        # In Channels = 3, Out Channels = 6, Kernel Size = 5
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    
    def forward(self, x):
        batch_size = 16 * 5 * 5
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, batch_size)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
net = Net()

In [5]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss() 
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [6]:
# This function is used for saving the current checkpoint after every epoch and also
# saving it as the best model if it is the best. 
def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')

In [12]:
def train(epochs): 
    for epoch in range(2): 
        running_loss = 0.0
        biggest_loss = 0
        best_loss = 1000000
        for i, data in enumerate(trainloader, 0): 
            inputs, labels = data
            
            # Checking to see if we are currently using a GPU or not
            if use_gpu:
                inputs = Variable(inputs.cuda())
                labels = Variable(labels.cuda())
            else:
                inputs, labels = Variable(inputs), Variable(labels)
                
            optimizer.zero_grad()

            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.data[0]
            if i % 2000 == 1999: 
                print('[%d, %5d] loss: %.3f' % 
                     (epochs + epoch + 1, i + 1, running_loss / 2000))
                
                # Finding the biggest loss by baches of 2000 
                biggest_loss = max((running_loss / 2000), biggest_loss)
                #if ((running_loss / 2000) > biggest_loss):
                #   biggest_loss = running_loss / 2000
                running_loss = 0.0
                
        # Saving checkpoint of model after every epoch so we can use it later 
        is_best = biggest_loss < best_loss
        best_loss = max(biggest_loss, best_loss)
        save_checkpoint({
            'epoch': epochs + epoch + 1,
            'state_dict': net.state_dict(),
            'best_loss': best_loss,
            'optimizer' : optimizer.state_dict(),
        }, is_best)

    print('Finished Training')

Starts from the last checkpoint. Allows for the checkpoint to be saved from training on a GPU and continue training on a CPU. 

In [13]:
model = Net()
resume = True 
file_checkpoint = 'checkpoint.pth.tar'
file_best = 'model_best.pth.tar'

if resume:
        if os.path.isfile(file_checkpoint):
            print("=> loading checkpoint '{}'".format(file_checkpoint))
            # This special torch.load allows to go from GPU to CPU
            checkpoint = torch.load(file_checkpoint, map_location=lambda storage, loc: storage)
            start_epoch = checkpoint['epoch']
            best_loss = checkpoint['best_loss']
            model.load_state_dict(checkpoint['state_dict'])
            optimizer.load_state_dict(checkpoint['optimizer'])
            print("=> loaded checkpoint '{}' (epoch {})"
                  .format(file_checkpoint, checkpoint['epoch']))
        else:
            print("=> no checkpoint found at '{}'".format(file_checkpoint))
            
train(start_epoch)

=> loading checkpoint 'checkpoint.pth.tar'
=> loaded checkpoint 'checkpoint.pth.tar' (epoch 8)
[9,  2000] loss: 0.887
[9,  4000] loss: 0.880
[9,  6000] loss: 0.900
[9,  8000] loss: 0.916
[9, 10000] loss: 0.915
[9, 12000] loss: 0.917
[10,  2000] loss: 0.833
[10,  4000] loss: 0.822
[10,  6000] loss: 0.877
[10,  8000] loss: 0.881
[10, 10000] loss: 0.910
[10, 12000] loss: 0.884
Finished Training
